In [ ]:
import profit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
profit.Globals.GLOBAL_LEVEL = profit.Globals.LOG_WARNING

In [ ]:
# Input file
fname = "data/combined_genie_large.gump.df"

# Fit variable -- try out calorimetric neutrino energy, transverse momentum

# fitvar = "caloE"
# unit = "Reconstructed Neutrino Energy [GeV]"
# bins = [0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.25, 1.5, 2.0, 3.0]
# bins = list(np.linspace(0.3, 1.3, 11)) + [1.5, 2., 3.]

fitvar = "dPt"
unit = "Reconstructed Transverse Momentum [GeV/c]"
bins = np.linspace(0, 1, 11)

# Lookup POT
FILE_POT = pd.read_hdf(fname, key="pot")[0].values[0]
FILE_POT

In [ ]:
# CONFIGURE PROfit INSTANCE

from profit import xml as pxml

mode = "nu"
detector = "SBND"
POT = 2e20

numucc_subchannel = pxml.PROXMLSubChannel(
  name="numucc",
  plotname="#nu_{#mu} CC",
  color="#FF6961"
)

channels = [
  pxml.PROXMLChannel(
    name="numu",
    unit=unit,
    bins=bins,
    truebin_min=0,
    truebin_max=2,
    truebin_nbin=200,
    subchannel=[numucc_subchannel]
  )
]

flux_variations = [
    pxml.PROXMLVariation(name="expskin_Flux", type="covariance"),
    pxml.PROXMLVariation(name="horncurrent_Flux", type="covariance"),
    pxml.PROXMLVariation(name="nucleoninexsec_Flux", type="covariance"),
    pxml.PROXMLVariation(name="nucleonqexsec_Flux", type="covariance"),
    pxml.PROXMLVariation(name="nucleontotxsec_Flux", type="covariance"),
    pxml.PROXMLVariation(name="pioninexsec_Flux", type="covariance"),
    pxml.PROXMLVariation(name="pionqexsec_Flux", type="covariance"),
    pxml.PROXMLVariation(name="piontotxsec_Flux", type="covariance"),
    pxml.PROXMLVariation(name="piplus_Flux", type="covariance"),
    pxml.PROXMLVariation(name="piminus_Flux", type="covariance"),
    pxml.PROXMLVariation(name="kplus_Flux", type="covariance"),
    pxml.PROXMLVariation(name="kminus_Flux", type="covariance"),
    pxml.PROXMLVariation(name="kzero_Flux", type="covariance"),
]

xsec_variations = [
    pxml.PROXMLVariation(name="GENIEReWeight_SBN_v1_multisigma_RPA_CCQE", type="spline"),
    pxml.PROXMLVariation(name="GENIEReWeight_SBN_v1_multisigma_ZExpA1CCQE", type="spline"),
    pxml.PROXMLVariation(name="GENIEReWeight_SBN_v1_multisigma_ZExpA2CCQE", type="spline"),
    pxml.PROXMLVariation(name="GENIEReWeight_SBN_v1_multisigma_ZExpA3CCQE", type="spline"),
    pxml.PROXMLVariation(name="GENIEReWeight_SBN_v1_multisigma_ZExpA4CCQE", type="spline"),
]

syst_friends = [
  pxml.PROXMLFriendTree(filename=fname, treename="wgt"),
]

numucc_branch = pxml.PROXMLBranch(
  associated_subchannel=pxml.PROXMLBranch.get_associated_subchannel(mode, detector, channels[0], numucc_subchannel),
  name=fitvar,
  true_param_name="trueE",
  true_L_name="baseline / 1000",
  pdg_name="truepdg",
  additional_weight="(iscc == 1) && ((truepdg == 14) || (truepdg == -14))",
  model_rule="1"
)

mcfiles = [
  pxml.PROXMLMCFile(
    filename=fname,
    treename="var",
    pot=FILE_POT,
    friend=syst_friends,
    branch=[numucc_branch],
    maxevents=50000,
    scale=1.0,
  )
]

rules = [
  pxml.PROXMLModelRule(name="No Osc", index=0),
  pxml.PROXMLModelRule(name="Numu Dis", index=1),
  pxml.PROXMLModelRule(name="Nue App", index=2),
]

model = pxml.PROXMLModel(
  tag="numudis",
  rule=rules
)

config = pxml.PROXMLMaker(
  mode=mode,
  detector=detector,
  model=model,
  channel=channels,
  plotpot=POT,
  allow_variation_list=xsec_variations,
  mcfile=mcfiles
)

with open("PROmcmc.xml", "w+") as f:
    f.write(config.to_xml())

print(config.to_xml())

In [ ]:
# Init the config object
c = profit.PROconfig("PROmcmc.xml")

# Load the SystStruct's and PROpeller
#
# This can be used interchangably with profit.PROcess_CAFana
syst_structs, prop = profit.PROcess_dataframes(c)

# Load systematic weights into PROsyst
systs = profit.PROsyst(syst_structs)

# Define oscillation model
osc = profit.PROsc(prop)

In [ ]:
# CV spectrum
data = profit.FillCVSpectrum(c, prop, True)

In [ ]:
bins = np.array(c.m_channel_bin_edges[0])
centers = (bins[:-1] + bins[1:]) / 2
widths = (bins[1:] - bins[:-1])

_ = plt.hist(centers, bins=bins, weights=data.Spec()/(widths/0.05), histtype="step", label="CV", linewidth=2)

# Example shifted spectrum
_ = plt.hist(centers, bins=bins, weights=profit.FillRecoSpectra(c, prop, systs, {"GENIEReWeight_SBN_v1_multisigma_RPA_CCQE": 1.}, True).Spec()/(widths/0.05),
            histtype="step", label="RPA $+1\\sigma$", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=profit.FillRecoSpectra(c, prop, systs, {"GENIEReWeight_SBN_v1_multisigma_ZExpA1CCQE": 1.}, True).Spec()/(widths/0.05),
            histtype="step", label="Z Exp. A1 $+1\\sigma$", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=profit.FillRecoSpectra(c, prop, systs, {"GENIEReWeight_SBN_v1_multisigma_ZExpA2CCQE": 1.}, True).Spec()/(widths/0.05),
            histtype="step", label="Z Exp. A2 $+1\\sigma$", linewidth=2)


plt.xlabel(c.m_channel_units[0])
plt.ylabel("Entries / 50 MeV / %.0e POT" % c.m_plot_pot)
plt.legend()

In [ ]:
# Get the chi2 function
osc_params = np.array([]) # No oscillation
nparams = systs.GetNSplines() # Number of parameters in fit. One for each systematic

chi = profit.PROchi("3plus1", c, prop, systs, osc, data, nparams, nparams, profit.EvalStrategy.BinnedChi2, osc_params)

In [ ]:
# bound fit at +/- 3 sigma
lower_bound = np.full((nparams,), -3)
upper_bound = np.full((nparams,), 3)

# parameters
param = profit.LBFGSBParam()
param.epsilon = 1e-6
param.max_iterations = 100
param.max_linesearch = 50
param.delta = 1e-6

fitter = profit.PROfitter(upper_bound, lower_bound, param)

In [ ]:
fit_result = fitter.Fit(chi)

In [ ]:
fitter.best_fit, fitter.Covariance(), np.sqrt(np.diag(fitter.Covariance()))

In [ ]:
# Run MCMC

In [ ]:
def log_probability(p):
    # Limits
    chi_v = chi(p, False)
    if np.isnan(chi_v):
        print(p)
    return -chi_v/ 2

In [ ]:
chi([0.]*nparams, False)

In [ ]:
import emcee

ndim = nparams
nwalkers = nparams*10
p0 = np.random.randn(ndim * nwalkers).reshape((nwalkers, ndim)) # Initial positions of walkers
nsteps = 10_000  # Number of steps

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability)
_ = sampler.run_mcmc(p0, nsteps, progress=True)

In [ ]:
samples = sampler.get_chain(flat=True, thin=100)

In [ ]:
np.mean(samples, axis=0), np.std(samples, axis=0)

In [ ]:
fitter.BestFit(), np.sqrt(np.diag(fitter.Covariance()))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
_ = plt.hist(samples[:, 0])

In [ ]:
import corner

In [ ]:
_ = corner.corner(samples, truths=[0]*5, quantiles=[0.5 - 0.34, 0.5 + 0.34], range=[(-3, 3)]*5)